Set Up

In [99]:
import falcon
import ntt

from common import q
from numpy import set_printoptions
from math import sqrt
from fft import fft, ifft, sub, neg, add_fft, mul_fft
from ntt import sub_zq, mul_zq, div_zq
from ffsampling import gram, ffldl_fft, ffsampling_fft
from ntrugen import ntru_gen
from encoding import compress, decompress
# https://pycryptodome.readthedocs.io/en/latest/src/hash/shake256.html
from Crypto.Hash import SHAKE256, SHA512
# Randomness
from os import urandom
from rng import ChaCha20
# For debugging purposes
import sys
if sys.version_info >= (3, 4):
    from importlib import reload  # Python 3.4+ only.

set_printoptions(linewidth=200, precision=5, suppress=True)

logn = {
    2: 1,
    4: 2,
    8: 3,
    16: 4,
    32: 5,
    64: 6,
    128: 7,
    256: 8,
    512: 9,
    1024: 10
}

# Bytelength of the signing salt and header
HEAD_LEN = 1
SALT_LEN = 40
SEED_LEN = 56

# Parameter sets for Falcon:
# - n is the dimension/degree of the cyclotomic ring
# - sigma is the std. dev. of signatures (Gaussians over a lattice)
# - sigmin is a lower bounds on the std. dev. of each Gaussian over Z
# - sigbound is the upper bound on ||s0||^2 + ||s1||^2
# - sig_bytelen is the bytelength of signatures
Params = {
    # FalconParam(2, 2)
    2: {
        "n": 2,
        "sigma": 144.81253976308423,
        "sigmin": 1.1165085072329104,
        "sig_bound": 101498,
        "sig_bytelen": 44,
    },
    # FalconParam(4, 2)
    4: {
        "n": 4,
        "sigma": 146.83798833523608,
        "sigmin": 1.1321247692325274,
        "sig_bound": 208714,
        "sig_bytelen": 47,
    },
    # FalconParam(8, 2)
    8: {
        "n": 8,
        "sigma": 148.83587593064718,
        "sigmin": 1.147528535373367,
        "sig_bound": 428865,
        "sig_bytelen": 52,
    },
    # FalconParam(16, 4)
    16: {
        "n": 16,
        "sigma": 151.78340713845503,
        "sigmin": 1.170254078853483,
        "sig_bound": 892039,
        "sig_bytelen": 63,
    },
    # FalconParam(32, 8)
    32: {
        "n": 32,
        "sigma": 154.6747794602761,
        "sigmin": 1.1925466358390344,
        "sig_bound": 1852696,
        "sig_bytelen": 82,
    },
    # FalconParam(64, 16)
    64: {
        "n": 64,
        "sigma": 157.51308555044122,
        "sigmin": 1.2144300507766141,
        "sig_bound": 3842630,
        "sig_bytelen": 122,
    },
    # FalconParam(128, 32)
    128: {
        "n": 128,
        "sigma": 160.30114421975344,
        "sigmin": 1.235926056771981,
        "sig_bound": 7959734,
        "sig_bytelen": 200,
    },
    # FalconParam(256, 64)
    256: {
        "n": 256,
        "sigma": 163.04153322607107,
        "sigmin": 1.2570545284063217,
        "sig_bound": 16468416,
        "sig_bytelen": 356,
    },
    # FalconParam(512, 128)
    512: {
        "n": 512,
        "sigma": 165.7366171829776,
        "sigmin": 1.2778336969128337,
        "sig_bound": 34034726,
        "sig_bytelen": 666,
    },
    # FalconParam(1024, 256)
    1024: {
        "n": 1024,
        "sigma": 168.38857144654395,
        "sigmin": 1.298280334344292,
        "sig_bound": 70265242,
        "sig_bytelen": 1280,
    },
}

In [100]:
import random

def seeded_rng(i):
    random_generator = random.Random(42)
    return bytes(random_generator.getrandbits(8) for _ in range(i))

def zero_rng(i):
    return b'\x00'*i

In [130]:
n = 512
sk = falcon.SecretKey(n)
pk = falcon.PublicKey(sk)
byte_string = b'test'
signature = sk.sign(byte_string, seeded_rng)
sk.verify(byte_string, signature)

True

Public Key Recovery Operations

In [111]:
import numpy as np

In [145]:
all([0,1])

False

In [149]:
def sign_pk_recovery(sk, message, randombytes=urandom):
    """
    Sign a message. The message MUST be a byte string or byte array.
    Optionally, one can select the source of (pseudo-)randomness used
    (default: urandom).
    """
    int_header = 0x30 + logn[sk.n]
    header = int_header.to_bytes(1, "little")

    salt = randombytes(SALT_LEN)
    hashed = sk.hash_to_point(message, salt)

    # We repeat the signing procedure until we find a signature that is
    # short enough (both the Euclidean norm and the bytelength)
    while(1):
        if (randombytes == urandom):
            s = sk.sample_preimage(hashed)
        else:
            seed = randombytes(SEED_LEN)
            s = sk.sample_preimage(hashed, seed=seed)
        norm_sign = sum(coef ** 2 for coef in s[0])
        norm_sign += sum(coef ** 2 for coef in s[1])
        # Check the Euclidean norm
        if norm_sign > sk.signature_bound:
            continue
            
        # Check that s2 (s[1]) is invertible by seeing that the NTT format doesn't have 0 as coeff
        s2_ntt = ntt.ntt(s[1])
        if not all(s2_ntt):
            continue
        
        # signature is (compress(s1), compress(s2), r)
        enc_s1 = compress(s[0], sk.sig_bytelen - HEAD_LEN - SALT_LEN)
        enc_s2 = compress(s[1], sk.sig_bytelen - HEAD_LEN - SALT_LEN)
        
        
        # Check that the encoding is valid (sometimes it fails)
        if enc_s1 and enc_s2:
            return header + enc_s1 + enc_s2 + salt

In [150]:
def verify_pk_recovery(sk, pk, message, signature):
    """
    Verify a signature.
    """
    signature_length = sk.sig_bytelen - HEAD_LEN - SALT_LEN
    
    enc_s1 = signature[HEAD_LEN: signature_length + HEAD_LEN]
    enc_s2 = signature[signature_length + HEAD_LEN: 2*signature_length + HEAD_LEN]
    salt = signature[2*signature_length + HEAD_LEN:]
    
    # Need to unpack polynomial s1 and s2
    s1 = decompress(enc_s1, signature_length, sk.n)
    s2 = decompress(enc_s2, signature_length, sk.n)
    
    ## Check that s1 and s2 are valid
    if not s1 or not s2:
        print("Invalid encoding")
        return False

    # Check that the (s1, s2) is short
    norm_sign = sum(coef ** 2 for coef in s1)
    norm_sign += sum(coef ** 2 for coef in s2)
    if norm_sign > sk.signature_bound:
        print("Squared norm of signature is too large:", norm_sign)
        return False
    
    # Check that pk = H(inverse(s2)*(HashToPoint(r||m, q, n) - s1))
    
    hash_to_point_message = sk.hash_to_point(message, salt)
    recovered_pk = div_zq(sub_zq(hash_to_point_message, s1),s2)
    
    return recovered_pk == pk.h

The encoded values are concatenated into a bit sequence of 14n bits, which is then represented as ⌈14n/8⌉ bytes.

Display Correctness of public key recovery

In [152]:
for i in range(100):
    signature_pkr = sign_pk_recovery(sk, byte_string)
    if not verify_pk_recovery(sk, pk, byte_string, signature_pkr):
        print(f"Failed round{i}")

Additional Tests

In [72]:
n = 2
sk = falcon.SecretKey(n)
pk = falcon.PublicKey(sk)
byte_string = b'test'
signature = sk.sign(byte_string, seeded_rng)
sk.verify(byte_string, signature)

True

In [73]:
sk

Private key for n = 2:

f = [-24, 60]
g = [69, 32]
F = [-93, 20]
G = [-53, -66]

In [69]:
pk

Public for n = 2:

h = [3633, 11120]

Showing that the NTRU Equation Holds

In [76]:
t1 = sk.f[0]*sk.G[0]-sk.f[1]*sk.G[1]-sk.F[0]*sk.g[0]+sk.F[1]*sk.g[1]
print(t1, t1%12289)

12289 0


In [77]:
t2 = sk.f[0]*sk.G[1]+sk.f[1]*sk.G[0]-sk.F[1]*sk.g[0]-sk.F[0]*sk.g[1]
print(t2, t2%12289)

0 0


Showing that G - Fh = 0 mod phi mod q
- therefore B (A.T) = 0
- B = {{g, -f}, {G, -F}}
- A = {{1},{h}}
- What about A = {{-h, I},{qI,0}}?

In [80]:
t1 = sk.G[1]-sk.F[1]*pk.h[0]-sk.F[0]*pk.h[1]
print(t1, t1%12289)

454693 0


In [81]:
t2 = sk.G[0]-sk.F[0]*pk.h[0]+sk.F[1]*pk.h[1]
print(t2, t2%12289)

393248 0
